In [1]:
filepath = '/content/MyGoogleDrive/My Drive/iSpan finalreport/'
from google.colab import drive
drive.mount('/content/MyGoogleDrive')

Mounted at /content/MyGoogleDrive


###import files

In [8]:
import pandas as pd
df_original_1=pd.read_csv(filepath + "application_train.csv") #訓練資料集
df_original_2=pd.read_csv(filepath + "prev_app_final.csv")  #xinyi
df_original_3=pd.read_csv(filepath + "POS_CASH_balance_newCURR.csv") #昭哥
df_original_4=pd.read_csv(filepath + "new_credit_card_balance.csv") #Me
df_original_5=pd.read_csv(filepath + "instalment_final.csv") #思吟
df_original_6=pd.read_csv(filepath + "application_test.csv") #test

In [9]:
df1=df_original_1
df2=df_original_2
df3=df_original_3
df4=df_original_4
df5=df_original_5
df6=df_original_6

## observe data.shape

In [20]:
print(df1.shape,df2.shape,df3.shape,df4.shape,df5.shape,df6.shape)

(307511, 122) (338605, 31) (337224, 8) (103558, 13) (339587, 6) (48744, 121)


In [17]:
48744/307511 #訓練跟測試資料是8.5 1.5分

0.1585114028441259

##merge data to observe

In [38]:
# 原本merge報錯 以為是型態問題，後來抓到是cols名稱相同的不只一個
df2['SK_ID_CURR']=df2['SK_ID_CURR'].astype(int)
df1['SK_ID_CURR']=df1['SK_ID_CURR'].astype(int)
print(type(df2['SK_ID_CURR'][1]),type(df1["SK_ID_CURR"][1]))

<class 'numpy.int64'> <class 'numpy.int64'>


In [40]:
#df2['SK_ID_CURR']=df2['SK_ID_CURR'].astype(object)
df1_2=pd.merge(df1,df2,on="SK_ID_CURR",how="outer")
df1_3=pd.merge(df1,df3,how="outer") 
df1_4=pd.merge(df1,df4,how="outer")
df1_5=pd.merge(df1,df5,how="outer")

In [41]:
print(df1_2.shape,df1_3.shape,df1_4.shape,df1_5.shape)

(355286, 152) (355315, 129) (324164, 134) (355455, 127)


In [ ]:
df1_4.head()

,Unnamed: 0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_EDUCATION_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,AMT_CREDIT_LIMIT_ACTUAL_max,CNT_INSTALMENT_MATURE_CUM_max,SK_DPD_max,DRAWINGS_ATM_average,DRAWINGS_POS_average,DRAWINGS_average,AMT_PAYMENT_TOTAL_CURRENT_mean,AMT_INST_MIN_REGULARITY_mean,AMT_CREDIT_LIMIT_ACTUAL_mode,AMT_BALANCE_count
0,0.0,100002,1.0,0.0,202500.0,1.0,0.018801,-9461.0,-637.0,-3648.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,100003,0.0,0.0,270000.0,3.0,0.003541,-16765.0,-1188.0,-1186.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,100004,0.0,0.0,67500.0,1.0,0.010032,-19046.0,-225.0,-4260.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,100006,0.0,0.0,135000.0,1.0,0.008019,-19005.0,-3039.0,-9833.0,...,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,270000.0,0.0
4,4.0,100007,0.0,0.0,121500.0,1.0,0.028663,-19932.0,-3038.0,-4311.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##find null and observe with test.data

In [42]:
#把沒有在train的id都給抓出來
df1_2_mask=df1_2.TARGET.isnull()
df1_3_mask=df1_3.TARGET.isnull()
df1_4_mask=df1_4.TARGET.isnull()
df1_5_mask=df1_5.TARGET.isnull()
df1_2du=df1_2[df1_2_mask]
df1_3du=df1_3[df1_3_mask]
df1_4du=df1_4[df1_4_mask]
df1_5du=df1_5[df1_5_mask]

In [44]:
print(df1_2du.shape,df1_3du.shape,df1_4du.shape,df1_5du.shape)

(47775, 152) (47804, 129) (16653, 134) (47944, 127)


In [58]:
# 寫集合看一下 不再train的id是不是都在test裡面 A:是
set2=set([i for i in df1_2du['SK_ID_CURR']])
set3=set([i for i in df1_3du['SK_ID_CURR']])
set4=set([i for i in df1_4du['SK_ID_CURR']])
set5=set([i for i in df1_5du['SK_ID_CURR']])
set6=set([i for i in df6['SK_ID_CURR']])
print(len((set2&set6)),
      len((set3&set6)),
      len((set4&set6)),
      len((set5&set6)))

47775 47804 16653 47944


##merge all table and delete ID from test.data

In [102]:
df_all=pd.merge(df1,df2,on="SK_ID_CURR",how="outer")
df_all=pd.merge(df_all,df3,how="outer") 
df_all=pd.merge(df_all,df4,how="outer")
df_all=pd.merge(df_all,df5,how="outer")

In [103]:
df_all.shape

(355650, 176)

In [104]:
mask = ~df_all["SK_ID_CURR"].isin(df6["SK_ID_CURR"])

In [105]:
df_train_final=df_all[mask]

In [106]:
df_train_final.shape

(307511, 176)

In [107]:
df_allx=pd.merge(df6,df2,on="SK_ID_CURR",how="left")
df_allx=pd.merge(df_allx,df3,how="left") 
df_allx=pd.merge(df_allx,df4,how="left")
df_allx=pd.merge(df_allx,df5,how="left")

In [108]:
mask1 = ~df6["SK_ID_CURR"].isin(df_all["SK_ID_CURR"])

In [109]:
len(mask1)

48744

In [110]:
df_allx.shape

(48744, 175)

In [111]:
df_allx

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,DRAWINGS_average,AMT_PAYMENT_TOTAL_CURRENT_mean,AMT_INST_MIN_REGULARITY_mean,AMT_CREDIT_LIMIT_ACTUAL_mode,AMT_BALANCE_count,SK_NUM_PREV,NUM_INSTALMENT_VERSION,DAYS_LATE_PAYMENT,NAN,AMT_ARREARS
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,NaN,NaN,NaN,NaN,NaN,2.0,1.50,7.75,0.0,0.00
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,NaN,NaN,NaN,NaN,NaN,1.0,2.00,23.67,0.0,0.00
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,24847.826087,43629.903000,5884.273636,157500.0,6.0,4.0,1.50,13.41,0.0,5.00
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,2578.321538,6700.035732,7360.036714,225000.0,22.0,3.0,1.67,5.04,0.0,6.67
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,NaN,NaN,NaN,NaN,NaN,1.0,1.00,12.25,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,NaN,NaN,NaN,NaN,NaN,1.0,2.00,7.33,0.0,0.00
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,NaN,NaN,NaN,NaN,NaN,4.0,1.00,7.84,0.0,1.50
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,NaN,NaN,NaN,NaN,NaN,2.0,1.50,28.42,0.0,0.00
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,NaN,NaN,NaN,NaN,NaN,2.0,1.00,6.22,0.0,0.00


In [ ]:
df_allx.to_csv(filepath + "df_test_final")
df_train_final.to_csv(filepath + "df_train_final")

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,DRAWINGS_average,AMT_PAYMENT_TOTAL_CURRENT_mean,AMT_INST_MIN_REGULARITY_mean,AMT_CREDIT_LIMIT_ACTUAL_mode,AMT_BALANCE_count,SK_NUM_PREV,NUM_INSTALMENT_VERSION,DAYS_LATE_PAYMENT,NAN,AMT_ARREARS
32,100241,Cash loans,F,N,N,0,67500.0,431280.0,17797.5,360000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,101107,Cash loans,F,N,Y,1,157500.0,525735.0,48348.0,450000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
233,101578,Revolving loans,F,N,Y,0,153747.0,135000.0,6750.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
300,102070,Cash loans,F,N,Y,0,247500.0,1107612.0,47677.5,990000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,102441,Cash loans,M,N,Y,0,225000.0,162000.0,17577.0,162000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48446,453940,Revolving loans,M,N,N,1,225000.0,202500.0,10125.0,202500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48540,454578,Cash loans,F,N,N,0,148500.0,272520.0,21658.5,225000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48577,454841,Revolving loans,M,N,Y,0,180000.0,315000.0,15750.0,315000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48709,455958,Cash loans,F,Y,Y,0,180000.0,320922.0,36292.5,283500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
